In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup

import time

In [ ]:
def get_content_from_new_tab(driver,script: str,element_id: str):
    driver.execute_script(script)
    alert_obj = driver.switch_to.alert
    alert_obj.accept()
    driver.switch_to.window(driver.window_handles[-1])
    
    element = None
    while not element:
        try:
            element = driver.find_element_by_id(element_id)
        except:
            print('.',end='')
            time.sleep(1)

    table_content = element.get_attribute('innerHTML')
    driver.close()
    return table_content

def get_data(table_content,tag: str):
    soup = BeautifulSoup(table_content)    
    data_cols = []
    for table_data in soup.find_all(tag):
        data = table_data.text.strip(':').strip()
        if data:
            data_cols.append(data)
    return data_cols

In [ ]:
tipo_consulta = 'Repasse'
ano = '2019'
UF = 'BA'
UF_nome = 'Bahia'
programa = 'ALIMENTAÇÃO ESCOLAR'

form_input = {
    'cbTipoConsulta': tipo_consulta, # Tipo_de_OPC
    'cbAno': ano,                    # Ano Inicio Vigencia
    'cbPrograma': programa,          # Programa/Projeto
    'cbUf': UF_nome                  # UF
}

fname = 'PNAE'
for value in form_input.values():
    fname += '_' + value.replace(' ','_')
fname += '.csv'
print('Arquivo de saída:',fname)

In [ ]:
driver = webdriver.Firefox()

url = 'https://www.fnde.gov.br/sigpcadm/sistema.pu?operation=localizar'

query_str = f'#consulta=tipoConsulta%3D2%26anoConcessao%3D{ano}%26programaId%3DC7%26ufId%3D{UF}'

driver.get(url+query_str)
# time.sleep(5)

In [ ]:
## get current_page
element_id = 'ext-comp-1004' # número da página atual
curren_page_elm = driver.find_element_by_id(element_id)
i = 0
current_page = ''
while not current_page:
    current_page = curren_page_elm.get_attribute('value')
    print('.',end='')
    time.sleep(1)
current_page = int(current_page)
new_page = current_page

## get total_pages
element_id = 'ext-comp-1005' # total de páginas
element = driver.find_element_by_id(element_id)
total_pages = int(element.text.split()[-1])

scripts = []
while current_page <= total_pages:
    print(f'\n Analisando página {current_page} de {total_pages}',end='  ')
    element_id = 'ext-gen240' # tabela com resultados (body)
    element = driver.find_element_by_id(element_id)
    table_content = element.get_attribute('innerHTML')
    soup = BeautifulSoup(table_content)
    for a in soup.find_all('a'):
        href = a['href']
        if 'abrirNoPconline' in href:
            scripts.append(href)
    
    if current_page == total_pages:
        print(f'\n Foram lidos {len(scripts)} scripts.')
        break
    ## click to next page
    element_id = 'ext-gen265'  # botão "next page"
    element = driver.find_element_by_id(element_id)
    element.click()
    
    ## update current_page
    while new_page == current_page:
        time.sleep(1)
        print('.',end='')
        new_page = int(curren_page_elm.get_attribute('value'))
    current_page = new_page

In [ ]:
sep = ','

## get headers
driver.switch_to.window(driver.window_handles[0])

script = scripts[0]
element_id='div_direita'
content = get_content_from_new_tab(driver,script,element_id)
cols = get_data(content,tag='th')
csv_content = sep.join(cols) + '\r\n'

In [ ]:
## get data

for i,script in enumerate(scripts):
    print('\n iteração:',i,end='  ')
    driver.switch_to.window(driver.window_handles[0])
    element_id = 'div_direita'
    content = get_content_from_new_tab(driver,script,element_id)
    cols = get_data(content,tag='td')
    csv_content += sep.join(cols) + '\r\n'

In [ ]:
if scripts:
    scripts_file = 'scripts_' + fname.split('.')[0] + '.txt'
    f = open(scripts_file, "w")
    f.write('\r\n'.join(scripts))
    f.close()

f = open(fname, "w")
f.write(csv_content)
f.close()


In [ ]:
# scripts_file = 'scripts_' + fname.split('.')[0] + '.txt'
# f = open(scripts_file, "r")
# lines = f.readlines()
# f.close()
# '\r\n'.join([line.strip() for line in lines])